** internal notes for steps to get information from Yelp! that we want to achieve
1) done - load cities.csv we created into a list
2) done - load cuisines.csv we created into a list
3) in progress - prepare the API URL, KEY and initial parameters to make contact with the Yelp Source. 
4) create a loop to go through each city and record the responses from yelp into a primary data frame. 
5) create a loop to go thorugh each cuisine in each city and record the responses from yelp into a primary data frame. 
6) after the loading of the data, we will want to print the entire dataframe into a CSV.
7) test CSV to ensure that we have data representing different cities and cuisines as outlined in our list. 
8) list data sources and prepare to write down what were the parameters that we used and limitations of the yelp API. 

things I want to see: 
    id,
    alias,
    name,
    url, 
    review count, 
    rating, 
    coordinates (long/lat)
    price
    location
        address1
        city
        zip_Code
        country
        state
        display address
        phone


In [1]:
import csv, os, requests, time, pandas as pd, matplotlib.pyplot as plt
from config import *

In [2]:
cities_csv = os.path.join("cities.csv")
cities = []
with open(cities_csv, newline="") as csvfile:
    csvreader = csv.reader(csvfile, delimiter="|")
    for row in csvreader:
        cities.append(row[0])

cuisines_csv = os.path.join("cuisines.csv")
cuisines = []
with open(cuisines_csv, newline="") as csvfile:
    csvreader = csv.reader(csvfile, delimiter="|")
    for row in csvreader:
        cuisines.append(row[0])

In [3]:
cuisines

['American', 'Mexican']

In [4]:
cities

['San Francisco, CA', 'Oakland, CA']

In [5]:
url = api_url + search_path 
headers = {'Authorization': 'Bearer ' + ykey}
cuisine_data = []
for city in cities:
    for cuisine in cuisines: 
        #time.sleep(1000)
        params = {'term': cuisine, 'location': city, 'limit': 50}
        response = requests.get(url, headers=headers, params=params).json()
        for r in range(5):
            if len(response["businesses"]) != 0:
                capture_dict = {
                        "search_city": city,
                        "search_cuisine": cuisine,
                }  
                if 'id' in response['businesses'][r].keys():
                    capture_dict["y_id"] = response['businesses'][r]['id']
                else: 
                    capture_dict["y_id"] = 'NA'
                if 'name' in response['businesses'][r].keys():
                    capture_dict["name"] = response['businesses'][r]['name']
                else: 
                    capture_dict["name"] = 'NA'
                if 'image_url' in response['businesses'][r].keys():
                    capture_dict["image_url"] = response['businesses'][r]['image_url']
                else: 
                    capture_dict["image_url"] = 'NA'
                if 'url' in response['businesses'][r].keys():
                    capture_dict["url"] = response['businesses'][r]['url']
                else: 
                    capture_dict["url"] = 'NA'
                if 'is_closed' in response['businesses'][r].keys():
                    capture_dict["closed"] = response['businesses'][r]['is_closed']
                else: 
                    capture_dict["closed"] = 'Unknown'
                if 'review_count' in response['businesses'][r].keys():
                    capture_dict["review_count"] = response['businesses'][r]['review_count']
                else: 
                    capture_dict["review_count"] = 0
                if 'rating' in response['businesses'][r].keys():
                    capture_dict["rating"] = response['businesses'][r]['rating']
                else: 
                    capture_dict["rating"] = ''
                if 'price' in response['businesses'][r].keys():
                    capture_dict["price"] = response['businesses'][r]['price']
                else: 
                    capture_dict["price"] = ''
                if 'latitude' in response['businesses'][r]['coordinates'].keys():
                    capture_dict["lat"] = response['businesses'][r]['coordinates']['latitude']
                else: 
                    capture_dict["lat"] = ''
                if 'longitute' in response['businesses'][r]['coordinates'].keys():
                    capture_dict["long"] = response['businesses'][r]['coordinates']['longitude']
                else: 
                    capture_dict["long"] = ''
                if 'address1' in response['businesses'][r]['location'].keys():
                    capture_dict["add1"] = response['businesses'][r]['location']['address1']
                else: 
                    capture_dict["add1"] = ''
                if 'city' in response['businesses'][r]['location'].keys():
                    capture_dict["city"] = response['businesses'][r]['location']['city']
                else: 
                    capture_dict["city"] = ''
                if 'state' in response['businesses'][r]['location'].keys():
                    capture_dict["state"] = response['businesses'][r]['location']['state']
                else: 
                    capture_dict["state"] = ''
                if 'zip_code' in response['businesses'][r]['location'].keys():
                    capture_dict["zip_code"] = response['businesses'][r]['location']['zip_code']
                else: 
                    capture_dict["zip_code"] = ''
                if 'country' in response['businesses'][r]['location'].keys():
                    capture_dict["country"] = response['businesses'][r]['location']['country']
                else: 
                    capture_dict["country"] = ''
                if 'display_phone' in response['businesses'][r].keys():
                    capture_dict["disp_ph"] = response['businesses'][r]['display_phone']
                else: 
                    capture_dict["disp_ph"] = ''
                cuisine_data.append(capture_dict)

yelp_data = pd.DataFrame(cuisine_data)
yelp_data.head()

,search_city,search_cuisine,y_id,name,image_url,url,closed,review_count,rating,price,lat,long,add1,city,state,zip_code,country,disp_ph
0,"San Francisco, CA",American,c0WpyZFR3EoEBKcoY2LZ3Q,Marlowe,https://s3-media1.fl.yelpcdn.com/bphoto/4CskEp...,https://www.yelp.com/biz/marlowe-san-francisco...,False,2714,4.0,$$,37.778320,,500 Brannan St,San Francisco,CA,94107,US,(415) 777-1413
1,"San Francisco, CA",American,wxzx3duu1c0CcomzEOvfOQ,The Front Porch,https://s3-media1.fl.yelpcdn.com/bphoto/0fmMVy...,https://www.yelp.com/biz/the-front-porch-san-f...,False,2802,4.0,$$,37.743800,,65 29th St,San Francisco,CA,94110,US,(415) 695-7800
2,"San Francisco, CA",American,ykvr3ib1gHQHI4tznSn8iA,Wayfare Tavern,https://s3-media2.fl.yelpcdn.com/bphoto/_WGCYE...,https://www.yelp.com/biz/wayfare-tavern-san-fr...,False,3888,4.0,$$$,37.793948,,558 Sacramento St,San Francisco,CA,94111,US,(415) 772-9060
3,"San Francisco, CA",American,XQLmEdXoMzOpffwoFaBtaQ,Kitchen Story,https://s3-media3.fl.yelpcdn.com/bphoto/NFhzbx...,https://www.yelp.com/biz/kitchen-story-san-fra...,False,3292,4.0,$$,37.764235,,3499 16th St,San Francisco,CA,94114,US,(415) 525-4905
4,"San Francisco, CA",American,DL-2iaugPoULjvZ6pyNRAQ,Causwells,https://s3-media1.fl.yelpcdn.com/bphoto/TVoM-2...,https://www.yelp.com/biz/causwells-san-francis...,False,743,4.0,$$,37.800134,,2346 Chestnut St,San Francisco,CA,94123,US,(415) 447-6081


In [6]:
yelp_data.to_csv("yelpapidata.csv", index=False, header=True)